## Créer notre RNN pour faire la catégorisation des genres

In [ ]:
%cd /tf/notebooks/notebooks
%pip install pandas
%pip install scikit-learn
%pip install seaborn

In [ ]:
import pandas as pd

version = 3
df = pd.read_csv('clean_data/v{version}_audio_features_30s_equal.csv'.format(version=2))

In [ ]:
# Encode labels
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df['label_encoded'] = label_encoder.fit_transform(df['label'])

# Dictionnaire clef valeur du label_encoded:label
label_dict = dict(zip(label_encoder.transform(label_encoder.classes_), label_encoder.classes_))

In [ ]:
label_dict

In [ ]:
from sklearn.preprocessing import StandardScaler

Y = df['label_encoded']
X = df.drop(['label', 'label_encoded'], axis=1)

scaler = StandardScaler()
X = scaler.fit_transform(X)

# transform Y to a ndarray
Y = Y.to_numpy()

In [ ]:
type(X)

In [ ]:
type(Y)

In [ ]:
Y

In [ ]:
from sklearn.model_selection import train_test_split


X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.4, random_state=42)

# extract X_val and Y_val from X_test and Y_test (0.5)
X_val, X_test, Y_val, Y_test = train_test_split(X_test, Y_test, test_size=0.5, random_state=42)

# save train, val and test into a csv
pd.DataFrame(X_train).to_csv('clean_data/v{version}_X_train.csv'.format(version=version), index=False)
pd.DataFrame(X_val).to_csv('clean_data/v{version}_X_val.csv'.format(version=version), index=False)
pd.DataFrame(X_test).to_csv('clean_data/v{version}_X_test.csv'.format(version=version), index=False)
pd.DataFrame(Y_train).to_csv('clean_data/v{version}_Y_train.csv'.format(version=version), index=False)
pd.DataFrame(Y_val).to_csv('clean_data/v{version}_Y_val.csv'.format(version=version), index=False)
pd.DataFrame(Y_test).to_csv('clean_data/v{version}_Y_test.csv'.format(version=version), index=False)

In [ ]:
X_train.shape, X_val.shape, X_test.shape, Y_train.shape, Y_val.shape, Y_test.shape

In [ ]:
# Reshape for LSTM RNN
from keras.utils import to_categorical

number_labels = len(label_dict)

# X_train_traited = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
# X_test_traited = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))
# X_val_traited = X_val.reshape((X_val.shape[0], 1, X_val.shape[1]))

X_train_traited = X_train
X_test_traited = X_test
X_val_traited = X_val

Y_train_traited = to_categorical(Y_train, num_classes=number_labels)
Y_test_traited = to_categorical(Y_test, num_classes=number_labels)
Y_val_traited = to_categorical(Y_val, num_classes=number_labels)

In [ ]:
X_train_traited.shape, X_test_traited.shape, X_val_traited.shape, Y_train_traited.shape, Y_test_traited.shape, Y_val_traited.shape

In [ ]:
# Define RNN model
from tensorflow.keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Flatten
# model V1
# model = Sequential()
# model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
# model.add(Dense(32, activation='relu'))
# model.add(Dense(32, activation='relu'))
# model.add(Dense(number_labels, activation='softmax'))

# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# model V1
# model = Sequential()
# model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
# model.add(Dense(32, activation='relu'))
# model.add(Dropout(0.2))
# model.add(Flatten())
# model.add(Dense(32, activation='relu'))
# model.add(Dense(number_labels, activation='softmax'))

# model V3
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Flatten())

model.add(Dense(number_labels, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print(model.summary())

In [ ]:
model.fit(X_train_traited, Y_train_traited, epochs=250, batch_size=32, validation_data=(X_val_traited, Y_val_traited), verbose=1)

## Enregistrement du modèle

In [ ]:
# save model
model.save('./models/v{version}_model.h5'.format(version=version))

In [ ]:
loss, accuracy = model.evaluate(X_test_traited, Y_test_traited, verbose=1)
print(f'Accuracy model: {accuracy*100}%') 